# Circular hole probllem

In this notebook an analytic solution for a disk-shaped hole is constructed.

The domain defined as
$$
    \Omega = \{ x \in \mathbb{R^2} : 0 < a < |x| < A\}
$$
Let
$$
    \Gamma^r = \{ x \in \mathbb{R^2} : |x| = r\},
$$
then $\partial\Omega = \Gamma^a \cup \Gamma^A$.

## Poisson

Sought is the solution to the problem
$$
\begin{aligned}
    \nabla^2 u = 0, & \quad x \in \Omega, \\
    \frac{\partial u}{\partial r} = 0, & \quad x \in \Gamma^a, \\
    u = g, & \quad x \in \Gamma^A, \\
\end{aligned}
$$
where the Laplacian in polar coordinates is
$$
    \nabla^2 = \frac{\partial^2}{\partial r^2}
               + \frac{1}{r}\frac{\partial}{\partial r}
               + \frac{1}{r^2}\frac{\partial^2}{\partial \theta^2}
$$
and $g$ is chosen such that the solution is easy to construct :-)

In [1]:
from sympy import *
from util import lua_code

In [2]:
# Laplacian in polar coordinates
def laplacian(f):
    return f.diff(r, 2) + f.diff(r)/r + f.diff(t, 2)/r**2

Solution ansatz:

In [3]:
r, t = symbols('r theta')
a, B = symbols('a B')
u = r**2 * sin(2*t) + B * sin(2*t) / r**2
u

B*sin(2*theta)/r**2 + r**2*sin(2*theta)

Check Laplacian

In [4]:
simplify(laplacian(u))

0

Check inner circle boundary

In [5]:
u.diff(r).subs(r, a)

-2*B*sin(2*theta)/a**3 + 2*a*sin(2*theta)

Hence,

In [6]:
u = u.subs(B, a**4)
u.diff(r).subs(r, a)

0

We set $g=u$ on the outer circle boundary.

Gradient:

In [7]:
u_x = simplify(u.diff(r) * cos(t) - u.diff(t) * sin(t) / r)
u_x

-2*a**4*sin(3*theta)/r**3 + 2*r*sin(theta)

In [8]:
u_y = simplify(u.diff(r) * sin(t) + u.diff(t) * cos(t) / r)
u_y

2*a**4*cos(3*theta)/r**3 + 2*r*cos(theta)

## Elasticity

For a plane-strain problem, we assume there exists a biharmonic function $\phi$ such that [1]

\begin{align}
    2\mu u_r &= -\frac{\partial \phi}{\partial r} + (1-\nu)r\frac{\partial\psi}{\partial \theta} \\
    2\mu u_\theta &= -\frac{1}{r}\frac{\partial \phi}{\partial \theta} + (1-\nu)r^2\frac{\partial\psi}{\partial r} \\
    \sigma_{rr} &= \frac{1}{r}\frac{\partial \phi}{\partial r} +
    \frac{1}{r^2}\frac{\partial^2 \phi}{\partial \theta^2} \\
    \sigma_{\theta\theta} &= \frac{\partial^2 \phi}{\partial r^2} \\
    \sigma_{r\theta} &= -\frac{\partial }{\partial r} \left(
      \frac{1}{r}\frac{\partial \phi}{\partial\theta}
    \right) \\
\end{align}
where $\mu$ is the shear modulus, $\nu$ is Poisson's ratio, and $\phi$ and $\psi$ are related by
$$
\nabla^2\phi = \frac{\partial}{\partial r}\left(r\frac{\partial\psi}{\partial\theta}\right)
$$

Remark: Plane-stress is modelled in [1], therefore the formula are adjusted here for plane-strain.

[1] "Stress Singularites Resulting From Various Boundary Conditions in Angular Corners of Plates in Extension", M. L. Williams, 1952, Journal of Applied Mechanics, 19 (4), pp. 526-528.

Sought is the solution to the problem
$$
\begin{aligned}
    \sigma_{rr} = 0, & \quad x \in \Gamma^a, \\
    \sigma_{r\theta} = 0, & \quad x \in \Gamma^a, \\
    u_r = g_r, & \quad x \in \Gamma^A, \\
    u_\theta = g_\theta, & \quad x \in \Gamma^A, \\
\end{aligned}
$$
for $\phi,\psi$ such that the equations of elasticity are satisfied.

Solution ansatz (from J.R. Barber, Elasticity, 2002):

In [9]:
r, t = symbols('r theta')
a, S, mu, nu = symbols('a S mu nu')
A = S*a**2
B = -S*a**4/2
phi = -S * r**2 * sin(2*t) / 2 + A * sin(2*t) + B * sin(2*t) / r**2
phi

-S*a**4*sin(2*theta)/(2*r**2) + S*a**2*sin(2*theta) - S*r**2*sin(2*theta)/2

Check that phi is biharmonic.

In [10]:
laplacian(simplify(laplacian(phi)))

0

Compute stresses

In [11]:
srr = simplify(phi.diff(r) / r + phi.diff(t, 2) / r**2)
stt = simplify(phi.diff(t, 2))
srt = simplify(-(phi.diff(t) / r).diff(r))

Check boundary condition on inner circle.

In [12]:
(srr.subs(r, a), srt.subs(r, a))

(0, 0)

Find $\psi$ from compatbility condition, disregarding integration constants.

In [13]:
psi = integrate(integrate(simplify(laplacian(phi)), r) / r, t)
psi

-2*S*a**2*cos(2*theta)/r**2

Check compatbility condition.

In [14]:
simplify((r*psi.diff(t)).diff(r) - laplacian(phi))

0

Compute displacement

In [15]:
ur = simplify(-phi.diff(r) + (1 - nu) * r * psi.diff(t)) / (2*mu)
ut = simplify(-phi.diff(t) / r + (1 - nu) * r**2 * psi.diff(r)) / (2*mu)

In [16]:
ur

S*(-a**4 - 4*a**2*r**2*(nu - 1) + r**4)*sin(2*theta)/(2*mu*r**3)

In [17]:
lua_code(ur)

'(1.0/2.0)*S*(-a^4 - 4*a^2*r^2*(nu - 1) + r^4)*math.sin(2*theta)/(mu*r^3)'

In [18]:
ut

S*(a**4 - 4*a**2*nu*r**2 + 2*a**2*r**2 + r**4)*cos(2*theta)/(2*mu*r**3)

In [19]:
lua_code(ut)

'(1.0/2.0)*S*(a^4 - 4*a^2*nu*r^2 + 2*a^2*r^2 + r^4)*math.cos(2*theta)/(mu*r^3)'

We set $g_r=u_r$ and $g_\theta = u_\theta$ on the outer circle boundary.

Compute Jacobian

In [20]:
J = {}
J['ur_r'] = simplify(ur.diff(r))
J['ur_t'] = simplify(ur.diff(t))
J['ut_r'] = simplify(ut.diff(r))
J['ut_t'] = simplify(ut.diff(t))

In [21]:
for key, value in J.items():
    print(f'local {key} = {lua_code(value)}')

local ur_r = (1.0/2.0)*S*(3*a^4 + 4*a^2*nu*r^2 - 4*a^2*r^2 + r^4)*math.sin(2*theta)/(mu*r^4)
local ur_t = -S*(a^4 + 4*a^2*r^2*(nu - 1) - r^4)*math.cos(2*theta)/(mu*r^3)
local ut_r = (1.0/2.0)*S*(-3*a^4 + 4*a^2*nu*r^2 - 2*a^2*r^2 + r^4)*math.cos(2*theta)/(mu*r^4)
local ut_t = S*(-a^4 + 4*a^2*nu*r^2 - 2*a^2*r^2 - r^4)*math.sin(2*theta)/(mu*r^3)
